In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw
from IPython import display
from lib.viz import showarray

from stable_baselines import SAC
from stable_baselines.common.vec_env import VecNormalize
from stable_baselines.common.cmd_util import make_vec_env

import os, time, json, io
os.environ["MLFLOW_TRACKING_URI"] = "sqlite:///mlruns/db.sqlite"
import mlflow
mlflow_client = mlflow.tracking.MlflowClient()

from lib import eos

#from lib.eos import EyeOnStickEnv
#ENV = EyeOnStickEnv

from lib.eos2d import EyeOnStickEnv2D
ENV = EyeOnStickEnv2D

from lib.run import run_env_nsteps

import logging
logger = logging.getLogger()

In [ ]:
NJ = 3
model_name, model_version = f"eos.{NJ}J", 287 # 213 # 38
model, model_source = None, None

params = {'GEAR_FUNC_NOISE': 0}

lastrun_metrics = {}
lastrun_trajs = None
lastrun_gearfuncs = None

fig, ax = plt.subplots(figsize=(4, 4), dpi=224/4)
colors="rgb"

while True:
    if model_version is not None:
        registered_model = mlflow_client.get_model_version(model_name, model_version)
    else:
        registered_model = mlflow_client.get_latest_versions(model_name, stages=["None"])[0]
        
    #if model_source is None or model_source != registered_model.source:
    model_source = registered_model.source
    actual_model_version = registered_model.version
    model = SAC.load(model_source)
    
    params_fname = f'{model_source}.json'
    with open(params_fname, 'r') as fp:
        loaded_params = json.load(fp)
        
    params = {**loaded_params, **params} # merge, overriding loaded params 
        
    env = make_vec_env(lambda: ENV(NJ, params), n_envs=1)
    model.set_env(env)
    env.env_method('set_render_info', {'name': model_name, 'version': model_version, 'real_version': actual_model_version})

    def get_metrics_dashboard(metrics):
        dashboard_img = Image.new('RGB', (224, 224))
        dashboard_draw = ImageDraw.Draw(dashboard_img)
        
        def draw_text(txt, vpos=0):
            vpos += 1
            dashboard_draw.text((10, 10*vpos), txt)
            return vpos

        # draw metrics
        vpos = 0
        for key, val in metrics.items():
            vpos = draw_text(f'{key:15s} {val:+.4f}', vpos=vpos)
        
        return np.asarray(dashboard_img)
    
    def get_episode_dashboard1(gearfuncs, trajs):
        if gearfuncs is None:
            return np.zeros((224, 224, 3))

        ax.cla()
        ax.set_xlim([-eos.PHI_AMP, eos.PHI_AMP])
        ax.set_ylim([-eos.PHI_AMP, eos.PHI_AMP])
    
        gfx = np.linspace(-eos.PHI_AMP, eos.PHI_AMP, 50)
        for i, gf in enumerate(gearfuncs):
            gfy = gf(gfx)
            ax.scatter(gfx, gfy, 1, colors[i])

        for i in range(trajs.shape[2]):
            ax.scatter(trajs[:,0,i], trajs[:,1,i], 1, colors[i])
        
        io_buf = io.BytesIO()
        fig.savefig(io_buf, format='raw')
        io_buf.seek(0)
        img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                             newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
        io_buf.close()

        return img_arr[:,:,[0,1,2]] # take only RGB from RGBA
    
    def get_episode_dashboard2(trajs):
        if trajs is None:
            return np.zeros((224, 224, 3))

        ax.cla()
        ax.autoscale(True)
        for i in range(trajs.shape[2]):
            ax.plot(range(trajs.shape[0]), trajs[:,0,i], colors[i])

        io_buf = io.BytesIO()
        fig.savefig(io_buf, format='raw')
        io_buf.seek(0)
        img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                             newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
        io_buf.close()

        return img_arr[:,:,[0,1,2]] # take only RGB from RGBA
    
    def get_episode_dashboard3(trajs):
        if trajs is None:
            return np.zeros((224, 224, 3))

        ax.cla()
        ax.autoscale(True)
        for i in range(trajs.shape[2]):
            ax.scatter(trajs[:,0,i], trajs[:,2,i], 1, colors[i]) # ... phi, dphi ...
        ax.axhline(y=0.0, color='grey', linestyle='-')

        io_buf = io.BytesIO()
        fig.savefig(io_buf, format='raw')
        io_buf.seek(0)
        img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                             newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
        io_buf.close()

        return img_arr[:,:,[0,1,2]] # take only RGB from RGBA
        
#    def get_blank_dashboard():
#        dashboard_img = Image.new('RGB', (224, 224))
#        dashboard_draw = ImageDraw.Draw(dashboard_img)
#        return np.asarray(dashboard_img)

    def get_episode_dashboard(lastrun_gearfuncs, lastrun_trajs):
        return np.hstack((
            get_episode_dashboard1(lastrun_gearfuncs, lastrun_trajs),
            get_episode_dashboard2(lastrun_trajs),
            get_episode_dashboard3(lastrun_trajs)
            #get_metrics_dashboard(lastrun_metrics)
        ))
                
    # run the episode, collect images of rendered environment and trajectory data
    # FIXME: era_env_render = np.zeros((params['MAX_NSTEPS'], eos.SCREEN_SIZE[0], eos.SCREEN_SIZE[1], 3))
    lastrun_env_images = None
    def displayfunc(img_array):
        global lastrun_env_images
        
        img_array = np.hstack((img_array, env.envs[0].render_step_dashboard())) # FIXME
        img_array = img_array[np.newaxis,...] # make it (1, SCREEN_SIZE-X, SCREEN_SIZE-y, 3)
        if lastrun_env_images is None:
            lastrun_env_images = img_array
        else:
            lastrun_env_images = np.vstack((lastrun_env_images, img_array))
        # (?, SCREEN_SIZE-X, SCREEN_SIZE-y)

    lastrun_trajs = None
    def trajfunc(traj): # (3, NJ) where 3 corresponds to phi and _phi and dphi
        global lastrun_trajs
        traj = traj[np.newaxis,:,:] # make it (1, 2, NJ)
        if lastrun_trajs is None:
            lastrun_trajs = traj
        else:
            lastrun_trajs = np.vstack((lastrun_trajs, traj))
        # (?, 2, NJ)

    logger.debug(f"run_env_nsteps: n_steps={params['MAX_NSTEPS']}")
    lastrun_metrics, data = run_env_nsteps(env, model, params['MAX_NSTEPS'], displayfunc=displayfunc, trajfunc=trajfunc)
    lastrun_gearfuncs = list(env.envs[0].gearfuncs) # FIXME
    
    episode_dashboard_img_array = get_episode_dashboard(lastrun_gearfuncs, lastrun_trajs)
    episode_dashboard_img_array = get_episode_dashboard(lastrun_gearfuncs, lastrun_trajs)
        
    for i in range(params['MAX_NSTEPS']):
        display.clear_output(wait=True)

        #img_array = np.hstack(( # , episode_dashboard_img_array))
        img_array = np.vstack((lastrun_env_images[i], episode_dashboard_img_array))
        showarray(img_array)
        time.sleep(0.1)